# Riddler Express

https://fivethirtyeight.com/features/so-you-want-to-tether-your-goat-now-what/

> From Luke Robinson, a serenading stumper:

> My daughter really likes to hear me sing “The Unbirthday Song” from “Alice in Wonderland” to her. She also likes to sing it to other people. Obviously, the odds of my being able to sing it to her on any random day are 364 in 365, because I cannot sing it on her birthday. The question is, though, how many random people would she expect to be able to sing it to on any given day before it became more likely than not that she would encounter someone whose birthday it is? In other words, what is the expected length of her singing streak?

Let's use a **R** simulation to calculate the approximate expected length of the singing streak.

## Naive implementation with `sample`

Here is a straightforward naive implementation that someone new to R might write.

In [1]:
trial1 <- function() {
  days <- 1:365
  n <- 0
  singing <- TRUE
  while (singing) {
    if (sample(days, 1, replace = TRUE) == 1) {
      singing <- FALSE
    } else {
      n <- n + 1
    }
  }
  n
}

That code performs one trial and returns the number people the girl sings to until she finds one whose birthday is today. We arbitrarily assume that today is day 1.

We're not even going to run this one, because it takes a little over 27 minutes to do 1 million trials.

## Naive implementation with `sample.int`

We can make it faster by calling `sample.int` instead of `sample`, something even an experienced R user might not think of right away. Note that we also change `days` to an integer instead of a range because that's what `sample.int` expects.

In [2]:
trial2 <- function() {
  days <- 365
  n <- 0
  singing <- TRUE
  while (singing) {
    if (sample.int(days, 1, replace = TRUE) == 1) {
      singing <- FALSE
    } else {
      n <- n + 1
    }
  }
  n
}

We're not going to run this one either, because it still takes over 18 minutes to do 1 million trials.

This is a big improvement, but it's still frustratingly slow.

## Why are these so slow?

These naive implementations are slow in R because the part that R has optimized to be fast is random sampling in `sample` and `sample.int`. `sample` is a wrapper around `sample.int`, and `sample.int` is [written in C](https://github.com/wch/r-source/blob/trunk/src/main/unique.c#L2127). Unfortunately, there is overhead associated with crossing the language boundary from R to C and back to R, and our naive implementations are doing it in a loop.

To gain some insight into this, take a look at the following comparison.

In [3]:
# sample 1 integer 1000 times
example1 <- function() {
    for (i in seq(1000)) {
        sample.int(365, 1, replace = TRUE)
    }
}

In [4]:
# sample 1000 integers 1 time
example2 <- function() {
    sample.int(365, 1000, replace = TRUE)
}

In [5]:
library(microbenchmark)

In [6]:
print(microbenchmark(example1(), example2()))

Unit: microseconds
       expr      min        lq       mean    median       uq      max neval
 example1() 1851.358 1951.1200 2356.02065 1991.3100 2655.490 5773.972   100
 example2()   36.979   40.1425   52.76395   40.8165   41.928 1135.201   100


It's *much* faster to do all of our sampling in one call instead of in a loop.

## How many samples do we need for each trial?

But to do it this way, we have to specify the number of samples up front. How many samples will we need for each trial?

The girl might encounter someone whose birthday is today right away, in which case we only need 1 sample.

Or she could go for a very long time without finding anyone whose birthday is today. In theory there is no upper limit, but we can use the rules of probability to figure out a reasonable upper bound on the number of samples we might need.

Suppose we are planning to run 1 million trials. Then an event that might occur only once in a million trials could realistically happen.

If we run 1 million trials, how long would we expect the most extreme singing streak to be?

Each time the girl approaches a new person, the probability that the person's birthday is **not** today is

$$
Prob(\texttt{birthday is not today}) = \frac{364}{365}
$$

The probability this would happen $n$ times in a row is

$$
Prob = \left(\frac{364}{365}\right)^n
$$

What value of $n$ will make this probability less than 1/1,000,000?

In other words, what value of $n$ would make this true?

$$
\left(\frac{364}{365}\right)^n < \left(\frac{1}{1,000,000}\right)
$$

Solving for $n$

$$
n \log \left(\frac{364}{365}\right) < \log \left(\frac{1}{1,000,000}\right)
$$

This is allowed because $log$ is a monotonically increasing function and both fractions are positive and therefore in the domain of $log$.

So $n$ needs to be at least 

$$
n > \frac{\log \left(\frac{1}{1,000,000}\right)}{\log \left(\frac{364}{365}\right)}
$$

The inequality flips because we divided both sides by $log \left(\frac{364}{365}\right)$ which is a negative number.

In [7]:
log(1 / 1e6) / log(364 / 365)

[1] 5035.75

To be safe, let's generate 6000 samples for each trial.

The probability that this will be too small for any given trial is

In [8]:
(364/365)^6000

[1] 7.097659e-08

which is about 1 in 700,000,000.

## Implementation using `purrr::detect_index`

We could do something like this using `purrr::detect_index` which returns the position of the first element for which a function returns `TRUE`. Notice that we're using R 4.1's new shorthand function syntax? Cool!

In [9]:
library(purrr)

trial3 <- function(n_samples) {
    detect_index(sample.int(365, 6000, replace = TRUE), \(x) x == 1) - 1
}

It turns out that this implementation isn't any faster. In fact it takes about 19 minutes to do 1 million trials, a minute or so longer than using `trial2` above.

I have no real idea why this is the case.

This implementation also has a subtle bug. `purrr::detect_index` returns 0 when it doesn't find a match, which in our case means that none of the 6000 samples had the value 1. We calculated that the probability of this is minuscule. We could probably just ignore this, but handling it correctly means adding a check for 0 on each trial, which would make the code even slightly slower.

## Old-school base R implementation

There is a way to speed up the simulation, and it uses old-school base R.

Here it is.

In [10]:
trial4 <- function() {
  which(sample.int(365, 6000, replace = TRUE) == 1)[1]
}

In [11]:
do_trials <- function(n_trials) {
  trials <- replicate(n_trials, trial4())
  mean(trials)
}

n_trials <- 1e6
print(paste("Performing", format(n_trials, big.mark = ",", scientific = FALSE), "trials..."))
system.time(print(do_trials(n_trials)))

[1] "Performing 1,000,000 trials..."
[1] 364.9035


   user  system elapsed 
293.213   0.107 293.436 

We're down to about 5 minutes, but that was a lot of work!

Also take a look at that R code for `trial4`. It might look perfectly reasonable to a seasoned R programmer, but newcomers and casual R users might struggle to read it, let alone develop it on their own.

### Addendum

Can we speed things up by running our trials in parallel?

Let's find out.

In [12]:
library(foreach)
library(doParallel)


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [13]:
parallel::detectCores()

[1] 4

My laptop only has 4 cores, and we'll leave one core free to take care of other tasks while the simulations are running.

In [14]:
n_cores <- parallel::detectCores() - 1
my_cluster <- parallel::makeForkCluster(n_cores)
my_cluster

socket cluster with 3 nodes on host ‘localhost’

In [15]:
doParallel::registerDoParallel(cl = my_cluster)

We'll continue to use `trial4` from above, but we'll run multiple copies in parallel.

In [16]:
do_trials <- function(n_trials) {
    replicate(n_trials, trial4())
}

n_trials <- 1e6

Since there are 3 nodes in our cluster, we'll run 1/3 of our trials on each.

In [17]:
do_all_trials <- function() {
    trials <- foreach(i = 1:3, .combine = 'c', .export = c("do_trials", "trial4")) %dopar% {
        do_trials(round(n_trials/3))
    }
    mean(trials)
}

print(paste("Performing", format(1e6, big.mark = ",", scientific = FALSE), "trials..."))
system.time(print(do_all_trials()))

[1] "Performing 1,000,000 trials..."
[1] 365.3402


   user  system elapsed 
  0.013   0.004 144.884 

In [18]:
doParallel::stopImplicitCluster()

That gives us a roughly 2x speedup, but not the 3x we might have hoped for because there is overhead associated with running simulations in parallel and safely combining the results.

So our fastest time for running 1 million simulations is now around 2.5 minutes, which is pretty good considering the first naive implementation took 27 minutes or so. But it was a lot of work to get that level of performance.

In [19]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 21.04

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.13.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] doParallel_1.0.16    iterators_1.0.13     foreach_1.5.1       
[4] purrr_0.3.4          microbenchmark_1.4-7

loaded via a namespace (and not attached):
 [1] codetools_0.2-18  fansi_0.5.0       digest_0.6.27     utf8_1.2.2       
 [5